Trazimo minimum neke funkcije f(x) pod uslovom da neko g(x) <= 17.

Optimizovanje F pod uslovom G1, G2, G3, ...

min f(x)
g_1(x) <= b_i
ili
-g_1 >= -b_i

f(x) = c*x = c0*x0 + c1*x1 + .... + cn*xn
g_i(x) = a_i*x

##### Linearno programiranje

min cx pod uslovom da vazi
Ax <= b

A - slozeni svi a_i u matricu
b - slozeni svi b_i u vektor

#### Zadatak

fiksni telefon - 12EUR
mobilni telefon - 20EUR profit

X - broj fiksnih
Y - broj mobilnih

max PROFIT x=? y=?

Trazimo max x\*12 + y\*20

1. Faza: fiksni sklapanje 0.2h, mobilni sklapanje 0.4h
2. Faza: zavrsni detalji: fiksni 0.5h, mobilni 0.4h

Ogranicenja: 
1. faza <= 400h
2. faza <= 490h

x >= 100
y >= 100

=> 

0.2\*X + 0.4\*y <= 400

0.5\*x + 0.4\*y <= 490

In [1]:
from docplex.mp.model import Model

In [2]:
model = Model(name='phones')

In [3]:
x = model.continuous_var(lb=100, name='x')

In [4]:
y = model.continuous_var(lb=100, name='y')

In [5]:
model.add_constraint(x>= 100)

docplex.mp.LinearConstraint[](x,GE,100)

In [6]:
model.add_constraint(y >= 100)

docplex.mp.LinearConstraint[](y,GE,100)

In [7]:
model.add_constraint(0.2*x + 0.4*y <= 400)

docplex.mp.LinearConstraint[](0.200x+0.400y,LE,400)

In [8]:
model.add_constraint(0.5*x + 0.4*y <= 490)

docplex.mp.LinearConstraint[](0.500x+0.400y,LE,490)

In [9]:
model.maximize(12 * x + 20 * y)

In [10]:
model.print_information()

Model: phones
 - number of variables: 2
   - binary=0, integer=0, continuous=2
 - number of constraints: 4
   - linear=4
 - parameters: defaults
 - objective: maximize
 - problem type is: LP


In [11]:
model.solve()

docplex.mp.solution.SolveSolution(obj=20600,values={x:300,y:850})

In [12]:
model.print_solution()

objective: 20600.000
status: OPTIMAL_SOLUTION(2)
  x=300.000
  y=850.000


UFLP
I korisnika
J resursa
Cij cena pridruzivanja korisnika i resursu j
fj cena resursa j

_Trazim: min{SUM(yj\*fj) + SUM(SUM(xij\*cij))}_

yj - 1 ako uspostavljamo, 0 ako ne uspostavljamo taj resurs

xij - 1 ako uspostavljamo s tim korisnikom, 0 ako ne 

**SUM(xij) - 1 za svako i (samo jednom resursu mozemo da pridruzimo korisnika)**

Ako je xij = 1 onda mora yj = 1

Ako je xij = 0 onda moze yj = 1 ili yj = 0

**xij <= yj za svako i i j**


In [13]:
def readInput(filename):
    with open(filename, 'r') as f:
        numUsers, numResources = [int(x) for x in f.readline().split()]
        cost = [[int(x) for x in f.readline().split()] for _ in range(numUsers)]
        fixedCost = [int(x) for x in f.readline().split()]
        
    return cost, fixedCost

In [14]:
cost, fixedCost = readInput('uflp1.txt')
numUsers = len(cost)
numResources = len(fixedCost)
print(numUsers, numResources)

3 3


In [15]:
model = Model(name='uflp')

In [16]:
x = model.binary_var_matrix(numUsers, numResources, name='X')

In [17]:
x

{(0, 0): docplex.mp.Var(type=B,name='X_0_0'),
 (0, 1): docplex.mp.Var(type=B,name='X_0_1'),
 (0, 2): docplex.mp.Var(type=B,name='X_0_2'),
 (1, 0): docplex.mp.Var(type=B,name='X_1_0'),
 (1, 1): docplex.mp.Var(type=B,name='X_1_1'),
 (1, 2): docplex.mp.Var(type=B,name='X_1_2'),
 (2, 0): docplex.mp.Var(type=B,name='X_2_0'),
 (2, 1): docplex.mp.Var(type=B,name='X_2_1'),
 (2, 2): docplex.mp.Var(type=B,name='X_2_2')}

In [18]:
y = model.binary_var_list(numResources, name='Y')

In [19]:
y

[docplex.mp.Var(type=B,name='Y_0'),
 docplex.mp.Var(type=B,name='Y_1'),
 docplex.mp.Var(type=B,name='Y_2')]

In [20]:
# SUM(xij) - 1 za svako i (samo jednom resursu mozemo da pridruzimo korisnika)

model.add_constraints([sum(x[i,j] for j in range(numResources)) == 1 for i in range(numUsers)] )

[docplex.mp.LinearConstraint[](X_0_0+X_0_1+X_0_2,EQ,1),
 docplex.mp.LinearConstraint[](X_1_0+X_1_1+X_1_2,EQ,1),
 docplex.mp.LinearConstraint[](X_2_0+X_2_1+X_2_2,EQ,1)]

In [21]:
# xij <= yj za svako i i j
for j in range(numResources):
    for i in range(numUsers):
        model.add_constraint(x[i, j] <= y[j])

In [22]:
model.print_information()

Model: uflp
 - number of variables: 12
   - binary=12, integer=0, continuous=0
 - number of constraints: 12
   - linear=12
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [23]:
# min{SUM(yj\*fj) + SUM(SUM(xij\*cij))}

# SUM(yj*fj)
first = 0
for j in range(numResources):
    first += y[j] * fixedCost[j]
    
# SUM(SUM(xij\*cij))

second = 0
for j in range(numResources):
    for i in range(numUsers):
         second += x[i, j] * cost[i][j]

In [24]:
model.minimize(first + second)

In [25]:
solution = model.solve()

In [26]:
solution.display()

solution for: uflp
objective: 34
status: OPTIMAL_SOLUTION(2)
X_0_0 = 1
X_1_0 = 1
X_2_0 = 1
Y_0 = 1


In [27]:
# I korisnika
# J resursa
# Cij, fj

# SSCLFP -> single source capacity limitation 

# Sj -> kapacitet resursa (koliko bolnica moze da stane u jedno naselje)
# di -> potraznja korisnika (koliko ljudi ima u naselju)

# minf = SUM(yj*fj) + SUM(SUM(xij*cij)) 

# SUM(xij) = 1  .

# xij <= yj 

# SUM(xij*di) <= Sj*yj - suma svih potraznji korisnika mora biti
# manja ili = kapacitetu uspostavljenog resursa


In [28]:
model = Model(name='ssclfp')

In [29]:
x = model.binary_var_matrix(numUsers, numResources, name='x')
y = model.binary_var_list(numResources, name='y')

In [30]:
model.add_constraints([sum(x[i,j] for j in range(numResources)) == 1 for i in range(numUsers)] )

[docplex.mp.LinearConstraint[](x_0_0+x_0_1+x_0_2,EQ,1),
 docplex.mp.LinearConstraint[](x_1_0+x_1_1+x_1_2,EQ,1),
 docplex.mp.LinearConstraint[](x_2_0+x_2_1+x_2_2,EQ,1)]

In [31]:
for i in range(numUsers):
    for j in range(numResources):
        model.add_constraint(x[i,j] <= y[j])

In [51]:
d = [1 for i in range(numUsers)]
s = [1.5 for j in range(numResources)]

In [33]:
for j in range(numResources):
    model.add_constraint(sum(x[i,j]*d[i] for i in range(numUsers)) <= s[j]*y[j])

In [34]:
model.print_information()

Model: ssclfp
 - number of variables: 12
   - binary=12, integer=0, continuous=0
 - number of constraints: 15
   - linear=15
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [35]:
first = sum(y[j]*fixedCost[j] for j in range(numResources))

In [36]:
second = sum(sum(x[i,j]*cost[i][j] for j in range(numResources)) for i in range(numUsers))

In [37]:
model.minimize(first + second)

In [38]:
model.solve()

docplex.mp.solution.SolveSolution(obj=34,values={x_0_0:1,x_1_0:1,x_2_0:1..

In [39]:
model.print_solution()

objective: 34
status: OPTIMAL_SOLUTION(2)
  x_0_0=1
  x_1_0=1
  x_2_0=1
  y_0=1


In [40]:
# MSCFLP:
    
# x[i,j] moze biti i 1/3 1/3 1/3 (zbir 1 ali nije 1 ili 0 vec xij E [0,1])
#

In [52]:
model = Model(name = 'mscflp')

In [53]:
x = model.continuous_var_matrix(numUsers, numResources, name='x')

In [54]:
y = model.binary_var_list(numResources, name='y')

In [55]:
model.add_constraints([sum(x[i,j] for j in range(numResources)) == 1 for i in range(numUsers)] )

[docplex.mp.LinearConstraint[](x_0_0+x_0_1+x_0_2,EQ,1),
 docplex.mp.LinearConstraint[](x_1_0+x_1_1+x_1_2,EQ,1),
 docplex.mp.LinearConstraint[](x_2_0+x_2_1+x_2_2,EQ,1)]

In [56]:
for i in range(numUsers):
    for j in range(numResources):
        model.add_constraint(x[i,j] <= y[j])

In [57]:
for j in range(numResources):
    model.add_constraint(sum(x[i,j]*d[i] for i in range(numUsers)) <= s[j]*y[j])

In [58]:
model.print_information()

Model: mscflp
 - number of variables: 12
   - binary=3, integer=0, continuous=9
 - number of constraints: 15
   - linear=15
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [59]:
first = sum(y[j]*fixedCost[j] for j in range(numResources))
second = sum(sum(x[i,j]*cost[i][j] for j in range(numResources)) for i in range(numUsers))
model.minimize(first+second)

In [60]:
model.solve()

docplex.mp.solution.SolveSolution(obj=36,values={x_0_0:0.5,x_0_2:0.5,x_1..

In [61]:
model.print_solution()

objective: 36.000
status: OPTIMAL_SOLUTION(2)
  x_0_0=0.500
  x_0_2=0.500
  x_1_0=1.000
  x_2_2=1.000
  y_0=1
  y_2=1


In [64]:
from scipy import optimize as opt
import numpy as np

In [ ]:
# min f(x1, x2) = 3x1 + 4x2
# x1 + 2x2 <= 14
# x2 <= 3x1
# x1 - x2 <= 2

In [66]:
C = np.array([3, 4])

A_ub = np.array([[1, 2],
                [-3, 1],
                [1, -1]])
b_ub = np.array([14, 0, 2])

In [68]:
opt.linprog(c=C, A_ub=A_ub, b_ub=b_ub)

       message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
       success: True
        status: 0
           fun: 0.0
             x: [ 0.000e+00  0.000e+00]
           nit: 0
         lower:  residual: [ 0.000e+00  0.000e+00]
                marginals: [ 3.000e+00  4.000e+00]
         upper:  residual: [       inf        inf]
                marginals: [ 0.000e+00  0.000e+00]
         eqlin:  residual: []
                marginals: []
       ineqlin:  residual: [ 1.400e+01  0.000e+00  2.000e+00]
                marginals: [-0.000e+00 -0.000e+00 -0.000e+00]